In [1]:
import PublicDataReader as pdr
from config import OpenAPI
import pandas as pd

# DataFrame 디스플레이 설정
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)
pd.set_option('display.width', 100)

print(pdr.__all__)
print(pdr.__version__)
# print(pdr.__info__)

['__version__', '__info__', 'Transaction', 'Building', 'StoreInfo', 'code_list']
2021.11.


In [2]:
serviceKey = OpenAPI['molit']
semas = pdr.StoreInfo(serviceKey, debug=True)

In [3]:
# 1. 지정상권
category = "지정상권"
key = "9174"

df = semas.read_data(category=category, key=key)

# 2. 반경상권
category = "반경상권"

radius = 500
cx = 127.03641615737838
cy = 37.50059843782878

df = semas.read_data(category=category, radius=radius, cx=cx, cy=cy)

# 3. 사각형상권
category = "사각형상권"

minx = 127.0327683531071
miny = 37.495967935149146
maxx = 127.04268179746694
maxy = 37.502402894207286

df = semas.read_data(category=category, minx=minx, miny=miny, maxx=maxx, maxy=maxy)

# 4. 행정구역상권
category = "행정구역상권"

divId = 'adongCd'
key = '1168058000'

df = semas.read_data(category=category,divId=divId, key=key)

# 5. 단일상가
category = "단일상가"

key = '11757465'

df = semas.read_data(category=category, key=key)

# 6. 건물상가
category = "건물상가"

key = '1168011000104940000004966'

df = semas.read_data(category=category, key=key)

# 7. 지번상가
category = "지번상가"

key = '1165010100108120002'
indsLclsCd = 'Q'

df = semas.read_data(category=category, key=key, indsLclsCd=indsLclsCd)

# 8. 행정동상가
category = "행정동상가"

divId = 'adongCd'
key = '1168064000'
indsLclsCd = 'Q'

df = semas.read_data(category=category, divId=divId, key=key, indsLclsCd=indsLclsCd)

# 9. 상권상가
category = "상권상가"

key = '9368'
indsLclsCd = 'Q'

df = semas.read_data(category=category, key=key, indsLclsCd=indsLclsCd)

# 10. 반경상가
category = "반경상가"

radius = '500'
cx = 127.03641615737838
cy = 37.50059843782878
indsLclsCd = 'Q'

df = semas.read_data(category=category, radius=radius, cx=cx, cy=cy, indsLclsCd=indsLclsCd)

# 11. 사각형상가
category = "사각형상가"

minx = 127.0327683531071
miny = 37.495967935149146
maxx = 127.04268179746694
maxy = 37.502402894207286
indsLclsCd = 'Q'

df = semas.read_data(category=category, minx=minx, miny=miny, maxx=maxx, maxy=maxy, indsLclsCd=indsLclsCd)

# 12. 다각형상가
category = "다각형상가"

key = 'POLYGON((127.02355609555755 37.504264372557095, 127.02496157306963 37.50590702991155, 127.0270858825753 37.50486867039889, 127.02628121988377 37.503489842823114))'
indsLclsCd = 'Q'

df = semas.read_data(category=category, key=key, indsLclsCd=indsLclsCd)

# 13. 업종별상가
category = "업종별상가"

divId = 'indsLclsCd'
key = 'Q'

df = semas.read_data(category=category, divId=divId, key=key)

# 14. 수정일자상가
category = "수정일자상가"

key = '20200101'
indsLclsCd = 'Q'

df = semas.read_data(category=category, key=key, indsLclsCd=indsLclsCd)

# 15. 업종대분류
category = "업종대분류"

df = semas.read_data(category=category, key=key)

# 16. 업종중분류
category = "업종중분류"

indsLclsCd = 'Q'

df = semas.read_data(category=category, key=key, indsLclsCd=indsLclsCd)

# 17. 업종소분류
category = "업종소분류"

indsLclsCd = 'Q'
indsMclsCd = 'Q01'

df = semas.read_data(category=category, key=key, indsLclsCd=indsLclsCd, indsMclsCd=indsMclsCd)


,trarNo,mainTrarNm,ctprvnCd,ctprvnNm,signguCd,signguNm,trarArea,coordNum,coords,stdrDt
0,9174,인사동,11,서울특별시,11110,종로구,226875,21,MULTIPOLYGON (((126.986059148338 37.5765234907...,2021-06-30


## Semas Class

## Transaction Class

In [ ]:
# 법정동코드 목록
code = pdr.code_list()
code.head()

In [20]:
# 공공 데이터 포털 Open API 서비스키
serviceKey = OpenAPI['molit']
self = pdr.Transaction(serviceKey, debug=True)

prod="아파트"
trans="매매"
sigunguCode="41135"
yearMonth="202101"

df = self.read_data(prod, trans, sigunguCode, yearMonth)
df2 = self.collect_data(prod, trans, sigunguCode, "202101", "202103")

[INFO] 아파트 매매 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 아파트 전월세 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 오피스텔 매매 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 오피스텔 전월세 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 단독다가구 매매 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 단독다가구 전월세 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 연립다세대 매매 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 연립다세대 전월세 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 상업업무용 매매 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 토지 매매 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 분양입주권 매매 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 아파트 매매 202101 조회 시작
[INFO] 아파트 매매 202102 조회 시작
[INFO] 아파트 매매 202103 조회 시작


## Building Class

In [ ]:
# 공공 데이터 포털 Open API 서비스키
serviceKey = OpenAPI['molit']
self = pdr.Building(serviceKey, debug=True)

category = "기본개요"

sigunguCd = "41135"
bjdongCd = "11000"
bun = "0560"
ji = "0003"

df = self.read_data(category, sigunguCd=sigunguCd, bjdongCd=bjdongCd, bun=bun, ji=ji)
df

## 컬럼 리스트 추출

In [ ]:
# LAWD_CD=11680
# DEAL_YMD=202110


# for prod in ['아파트','오피스텔','단독다가구','연립다세대','상업업무용','분양입주권','토지']:
#     for trans in ['매매', '전월세']:
#         try:
#             print()
#             print(f">> {prod} {trans}")
#             print()
#             service_url = self.metaDict[prod][trans]['url']

#             # URL
#             url=f"""\
# {service_url}\
# &LAWD_CD={str(LAWD_CD)}\
# &DEAL_YMD={str(DEAL_YMD)}\
# &numOfRows=99999\
#             """

#             result = requests.get(url, verify=False)
#             xmlsoup = BeautifulSoup(result.text, "lxml-xml")
#             item = xmlsoup.find("item")

#             col_list = []
#             for i in item:
#                 col_list.append(i.name)
#             for col in col_list: 
#                 print(f"'{col}'", end=',')
                
#         except:
#             print("Error")
#             pass

### Features

- 공통

    - 로깅 기능
        - 세션 초기화 시 `debug` 인자 추가
            - False(Default): 오류 레벨만 출력
            - True: 모든 레벨 출력
        - OPEN API 에러 코드 출력

    - Air Korea Open API 모듈 제거
    
    - 법정동코드 테이블 조회 함수 추가

- 부동산 실거래가

    - 데이터 조회 메서드 일원화
        - molit.read_data('아파트', '매매', '11680', '202111')
            - 서비스별 End-point 및 컬럼 매핑 딕셔너리 추가
    - 데이터 수집 메서드 신규 메서드 추가
        - molit.collect_data('아파트', '매매', '11680', '202101', '202103')

- 건축물대장
    
    - 데이터 조회 메서드 일원화
        - molit.read_data(category, sigunguCd=sigunguCd, bjdongCd=bjdongCd, bun=bun, ji=ji)